In [1]:
import polars as pl

import rustystats as rs

data = pl.read_parquet("https://raw.githubusercontent.com/PricingFrontier/pricing-data-example/917c853e256df8d5814721ab56f72889a908bb08/data/processed/frequency_set.parquet")

In [2]:
#creates analysis/exploration.json

exploration = rs.explore_data(
    data=data,
    response="ClaimCount",
    categorical_factors=["Region", "Area", "VehBrand", "VehGas"],
    continuous_factors=["VehPower", "VehAge", "DrivAge", "BonusMalus" ],
    exposure="Exposure",
)

In [3]:
# fits model

model = rs.glm(
    formula="ClaimCount ~ VehPower + VehAge + C(Area) + C(Region)",
    data=data,
    family="negbinomial",
    offset="Exposure"
).fit()

ModelDiagnostics(model_summary={'formula': 'ClaimCount ~ VehPower + VehAge + C(Area) + C(Region)', 'family': 'NegativeBinomial(theta=1.3802)', 'link': 'log', 'n_observations': 678012, 'n_parameters': 28, 'degrees_of_freedom_residual': 677984, 'converged': True, 'iterations': 6}, convergence_details=ConvergenceDetails(max_iterations_allowed=25, iterations_used=6, converged=True, reason='converged'), fit_statistics={'deviance': 196741.39765171794, 'null_deviance': 192800.37653563166, 'deviance_explained': -0.02044094097169946, 'log_likelihood': -98370.69882585897, 'aic': 196797.39765171794, 'bic': 197117.35141916262, 'pearson_chi2': 1678923.2261339806, 'dispersion_deviance': 0.29018590062850735, 'dispersion_pearson': 2.47634638300311}, loss_metrics={'family_deviance_loss': 0.2901740260018973, 'mse': 0.05512677026308909, 'mae': 0.09871081271150473, 'rmse': 0.2347909075392169}, calibration={'ae_ratio': 0.991, 'hl_pvalue': 0.0, 'problem_deciles': [{'decile': 0, 'ae': 6.44, 'n': 67124, 'ae_c

In [5]:
model = rs.glm(
    formula="ClaimCount ~ C(Area) + bs(VehAge, df=4)",
    data=data,
    family="negbinomial",
    offset="Exposure"
).fit()

In [2]:

model = rs.glm(
    formula="ClaimCount ~ C(Region) + VehPower + bs(VehAge, df=4) + bs(DrivAge, df=4) + BonusMalus + TE(Area) + TE(VehBrand)",
    data=data,
    family="negbinomial",
    offset="Exposure"
).fit()

In [3]:
print(model.summary())

                                 GLM Results                                  

Family:              NegativeBinomial(theta=0.0100) No. Observations:        678012
Link Function:       (default)       Df Residuals:            677981
Method:              IRLS            Df Model:                    30
Scale:               1.9267          Iterations:                   1

Log-Likelihood:                  nan Deviance:               1306237.4970
AIC:                             nan Null Deviance:            33408.8502
BIC:                             nan Pearson chi2:                    nan
Converged:           True           

------------------------------------------------------------------------------
Variable                             Coef    Std.Err        z    P>|z|                 95% CI     
------------------------------------------------------------------------------
Intercept                             nan  6293.0677      nan      nan   [     nan,      nan]     
Region[T.Aqu

In [5]:
#writes analysis/diagnostics
model.diagnostics(
    data=data,
    categorical_factors=["Region", "Area", "VehBrand", "VehGas"],
    continuous_factors=["VehPower", "VehAge", "DrivAge", "BonusMalus" ]
)

ValueError: cannot convert float NaN to integer